*NOTE*: this file is incorporated into a script in modelcmp_nonDL.py



# load data

In [1]:
import time
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import data as weardata
from wearables.scripts import eval_ as weareval

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import fastdtw
import lightgbm as lgb
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sktime.regression.compose._ensemble import ComposableTimeSeriesForestRegressor
from sktime.classification.compose import ComposableTimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


In [2]:
data = weardata.dataloader(target_name='GA')

In [3]:
# modify md
md = data.get_md_from_id(data.ids)
# add model outputs and errors here (add these fx to data loaders, since they handle the raw data slightly differently in future)
md = weardata.add_activitymetrics(data.data['data'], md)
md = weardata.calc_agg_metrics(md)

/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/pyActigraphy/metrics/metrics.py:44: RuntimeWarning: invalid value encountered in double_scalars
  return (d_24h / d_1h)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/pyActigraphy/metrics/metrics.py:54: RuntimeWarning: invalid value encountered in double_scalars
  return (c_1h / d_1h)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/pyActigraphy/metrics/metrics.py:583: RuntimeWarning: invalid value encountered in double_scalars
  return (m10-l5)/(m10+l5)


NaN in var: IS	n=IS
NaN in var: IV	n=IV
NaN in var: RA	n=RA
NaN in var: ISm	n=ISm
NaN in var: IVm	n=IVm
NaN in var: ave_logpseudocount_wake	n=ave_logpseudocount_wake


/home/ngrav/project/wearables/scripts/data.py:737: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  md = md.fillna(md.mean()) # fill with mean of column


In [5]:
voi = {
    'age_enroll': 'continuous',
    'GA': 'continuous',
    'gestage_by': 'categorical',
    'insur': 'categorical',
    'ethnicity': 'categorical',
    'bmi_1vis': 'continuous',
    'prior_ptb_all': 'categorical',
    'fullterm_births': 'categorical',
    'alcohol': 'categorical',
    'hypertension': 'categorical',
    'pregestational_diabetes': 'categorical',
    'asthma_yes___2': 'categorical', # diabetes
    'asthma_yes___8': 'categorical', # heart disease
    'asthma_yes___19': 'categorical', # depression
    'other_disease': 'categorical',
    'gestational_diabetes': 'categorical',
    'preeclampsia': 'categorical',
    'corticosteroids': 'categorical',
    'gyn_infection': 'categorical',
    'ptb_37wks': 'categorical',
    'epidural': 'categorical',
    'deliv_mode': 'categorical',
    'infant_wt': 'continuous',
    'infant_length': 'continuous',
    'head_circ': 'continuous',
    'death_baby': 'categorical',
    'neonatal_complication': 'categorical',
    'ervisit': 'categorical',
    'premature_birth1': 'categorical',
    'sleep_qual1': 'categorical',
    'slpenergy1': 'categorical',
    'IS': 'continuous',
    'IV': 'continuous',
    'RA': 'continuous',
    'min_rest': 'continuous',
    'ave_logpseudocount_wake': 'continuous',
    'ave_logpseudocount_sleep': 'continuous',
    'ave_logpseudocount_day': 'continuous',
    'ave_logpseudocount_night': 'continuous',
    'ave_logpseudocount_wknd': 'continuous',
    'ave_logpseudocount_wkday': 'continuous',
    # add custom ones 
    'PQSI': 'continuous',
    'KPAS': 'continuous',
    'EpworthSS': 'continuous',
    'Edinburgh': 'continuous',
  }

In [6]:
md['Pre-term birth'] = (md['ptb_37wks']==1)

In [7]:
summary = weardata.describe_from_md(md, voi, 'Pre-term birth', out_file='/home/ngrav/project/wearables/results/cohort_table_v522.csv')
summary

,Variable,All,Pre-term birth_False,Pre-term birth_True,Padj,Effect
age_enroll,age_enroll,29.00 (25.00 - 32.00),29.00 (25.00 - 32.00),29.00 (25.00 - 33.00),3.65e+00,3.04
GA,GA,19.00 (10.00 - 28.50),20.00 (11.00 - 30.00),16.00 (10.00 - 24.00),3.85e-03**,15.43
gestage_by=0.0,gestage_by=0.0,49 (1.99),49 (2.25),0 (0.00),2.42e-01,12.93 (False) -100.00 (True)
gestage_by=1.0,gestage_by=1.0,710 (28.83),611 (28.01),99 (35.11),1.09e+01,-2.82 (False) 21.78 (True)
gestage_by=2.0,gestage_by=2.0,1229 (49.90),1107 (50.76),122 (43.26),4.91e+02,1.72 (False) -13.30 (True)
...,...,...,...,...,...,...
ave_logpseudocount_wkday,ave_logpseudocount_wkday,2.44 (1.28 - 2.97),2.43 (1.25 - 2.96),2.52 (1.42 - 3.03),5.70e+00,2.33
PQSI,PQSI,10.00 (7.00 - 12.00),10.00 (7.00 - 12.00),10.00 (8.00 - 14.00),1.15e-02*,13.37
KPAS,KPAS,9.29 (7.82 - 10.50),9.28 (7.78 - 10.52),9.40 (8.12 - 10.38),2.56e+01,0.32
EpworthSS,EpworthSS,7.00 (3.00 - 11.00),7.00 (3.00 - 11.00),8.00 (4.00 - 12.00),2.99e-01,7.36


In [8]:
# save metadata since it's a bit slow
md.to_csv('/home/ngrav/project/wearables/results/md_v52_220124.csv')

In [9]:
data.data['cat_class_enc']

{'marital': {'0': 0.0, '1': 1.0},
 'gestage_by': {'-99.0': 0.0, '1.0': 1.0, '2.0': 2.0, '3.0': 3.0, '4.0': 4.0},
 'insur': {'-99.0': 0.0,
  '1.0': 1.0,
  '2.0': 2.0,
  '3.0': 3.0,
  '4.0': 4.0,
  '5.0': 5.0},
 'ethnicity': {'0': 0.0, '1': 1.0},
 'race': {'0': 0.0, '1': 1.0},
 'prior_ptb_all': {'0': 0.0, '1': 1.0},
 'fullterm_births': {'0': 0.0, '1': 1.0},
 'surghx_none': {'0.0': 0.0, '1.0': 1.0},
 'alcohol': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'smoke': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'drugs': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'hypertension': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'pregestational_diabetes': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'asthma_yes___1': {'0': 0.0, '1': 1.0},
 'asthma_yes___2': {'0': 0.0, '1': 1.0},
 'asthma_yes___3': {'0': 0.0, '1': 1.0},
 'asthma_yes___4': {'0': 0.0, '1': 1.0},
 'asthma_yes___5': {'0': 0.0, '1': 1.0},
 'asthma_yes___7': {'0': 0.0, '1': 1.0},
 'asthma_yes___8': {'0': 0.0, '1': 1.0},
 'asthma_yes___10': {'0': 0.0, '1': 1.0},
 'as

In [ ]:
# count in group
len(md.loc[md['Pre-term birth'], 'record_id'].unique())

# models dev

In [2]:
class kNNTS():
    '''
    Arguments:
      data (weardata class): dataloader class from data.py
    '''
    def __init__(self, n_trials=10, target_name='GA'):
        self.n_trials = n_trials
        self.target_name = target_name
    
        
    def load_data(self):
        return weardata.dataloader(target_name=self.target_name)
        
    def fit(self, data):
        if data.kfold > 1:
            cv_eval = {}
            for k, cv_fold in enumerate(data.Xy_train.keys()):
                print('    cv_fold: ', cv_fold)
                [(X_train, y_train), (X_val, y_val)] = data.Xy_train[cv_fold]
                X_train, X_val = from_2d_array_to_nested(X_train), from_2d_array_to_nested(X_val)
                knn = KNeighborsTimeSeriesClassifier(n_neighbors=5, distance="dtw", n_jobs=-1)
                knn.fit(X_train, y_train)
                eval_metrics = weareval.eval_output(knn.predict(X_val), y_val, tasktype=data.tasktype)
                cv_eval[cv_fold] = {'model': knn, 
                                    # 'data': [(X_train, y_train), (X_val, y_val)], # store just IDs?
                                    'metric': eval_metrics['mae'] if data.tasktype=='regression' else eval_metrics['balanced_acc_adj'],
                                    'metrics': eval_metrics}
            # retain only best model
            tmp = {cv_fold:cv_eval[cv_fold]['metric'] for cv_fold in cv_eval.keys()}
            bst_fold = min(tmp, key=tmp.get) if data.tasktype=='regression' else max(tmp, key=tmp.get)
            self.knn = cv_eval[bst_fold]['model']
            return {'model': self.knn, 'metrics': cv_eval[bst_fold]['metrics']}
        else:
            X_train, y_train = data.Xy_train
            X_val, y_val = data.Xy_val
            X_train, X_val = from_2d_array_to_nested(X_train), from_2d_array_to_nested(X_val)
            self.knn = knn = KNeighborsTimeSeriesClassifier(n_neighbors=5, distance="dtw", n_jobs=-1)
            self.knn.fit(X_train, y_train)
            eval_metrics = weareval.eval_output(self.knn.predict(X_val), y_val, tasktype=data.tasktype)
            return {'model': self.knn, 'metrics': eval_metrics}
    
    def eval_test(self, data):
        X_test, y_test = data.Xy_test
        X_test = from_2d_array_to_nested(X_test)
        eval_metrics = weareval.eval_output(self.knn.predict(X_test), y_test, tasktype=data.tasktype)
        return eval_metrics
    
    def del_model(self):
        del self.knn
    
    def run_trials(self, verbose=True):
        if verbose:
            t_start = time.time()
            print('Starting kNTimeSeries trials; predict {}'.format(self.target_name))
        results = {}
        for n in range(self.n_trials):
            if verbose:
                tic = time.time()
            data = self.load_data()
            self.fit(data)
            results[n] = self.eval_test(data)
            self.del_model()
            if verbose:
                print('  finished trial {} in {:.2f}-s\t{:.1f}-min elapsed'.format(n, time.time()-tic, (time.time()-t_start)/60))
        return results

In [ ]:
results = kNNTS(n_trials=2).run_trials()
results 

Starting kNTimeSeries trials; predict GA
    cv_fold:  0


/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1403: UserWarning: Cython DTW is deprecated from V0.10
  out[i, j] = metric(X[i], Y[j], **kwds)
IOStream.flush timed out
IOStream.flush timed out


In [8]:
results = pd.DataFrame(results).T
results['model'] = 'RandomForest'

In [9]:
results

,mae,mape,rho,P_rho,model
0,8.306864,0.579775,0.254155,1.120509e-08,RandomForest
1,8.332274,0.607974,0.221692,9.317345e-07,RandomForest
